In [1]:
!wget https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz

# Now, we just need to unzip that folder.
!tar -xvzf spark-3.0.0-bin-hadoop2.7.tgz
!pip install findspark


import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
import findspark
findspark.init()

--2024-06-23 16:07:39--  https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220272364 (210M) [application/x-gzip]
Saving to: ‘spark-3.0.0-bin-hadoop2.7.tgz’

spark-3.0.0-bin-had 100%[===================>] 210.07M  24.3MB/s    in 8.9s    

2024-06-23 16:07:49 (23.6 MB/s) - ‘spark-3.0.0-bin-hadoop2.7.tgz’ saved [220272364/220272364]

spark-3.0.0-bin-hadoop2.7/
spark-3.0.0-bin-hadoop2.7/NOTICE
spark-3.0.0-bin-hadoop2.7/kubernetes/
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-3.0.0-bin-hadoop2.7/kubernetes/dockerfiles/
spark-3.0.0-bin-hadoop2.7/k

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("RDD Demo").getOrCreate()

In [4]:
from pyspark.ml.feature import IDF, Tokenizer, HashingTF

In [5]:
data = [ (0, "This is the first document"),   (1, "This document is the second document"),
    (2, "And this is the third one"),   (3, "Is this the first document?"),
    (4, "The last document is the fifth one") ]

In [6]:
df = spark.createDataFrame(data, ["id", "text"])

In [9]:
df.show(truncate=False)

+---+------------------------------------+
|id |text                                |
+---+------------------------------------+
|0  |This is the first document          |
|1  |This document is the second document|
|2  |And this is the third one           |
|3  |Is this the first document?         |
|4  |The last document is the fifth one  |
+---+------------------------------------+



In [10]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")

In [11]:
word_df = tokenizer.transform(df)

In [14]:
word_df.show(truncate=False)

+---+------------------------------------+-------------------------------------------+
|id |text                                |words                                      |
+---+------------------------------------+-------------------------------------------+
|0  |This is the first document          |[this, is, the, first, document]           |
|1  |This document is the second document|[this, document, is, the, second, document]|
|2  |And this is the third one           |[and, this, is, the, third, one]           |
|3  |Is this the first document?         |[is, this, the, first, document?]          |
|4  |The last document is the fifth one  |[the, last, document, is, the, fifth, one] |
+---+------------------------------------+-------------------------------------------+



In [22]:
hashingtf = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=15)

In [23]:
hashing = hashingtf.transform(word_df)

In [24]:
hashing.show(truncate = False)

+---+------------------------------------+-------------------------------------------+--------------------------------------------+
|id |text                                |words                                      |rawFeatures                                 |
+---+------------------------------------+-------------------------------------------+--------------------------------------------+
|0  |This is the first document          |[this, is, the, first, document]           |(15,[2,3,4,10,13],[1.0,1.0,1.0,1.0,1.0])    |
|1  |This document is the second document|[this, document, is, the, second, document]|(15,[2,3,4,6,10],[1.0,1.0,1.0,1.0,2.0])     |
|2  |And this is the third one           |[and, this, is, the, third, one]           |(15,[0,1,2,3,4,6],[1.0,1.0,1.0,1.0,1.0,1.0])|
|3  |Is this the first document?         |[is, this, the, first, document?]          |(15,[1,2,3,4,13],[1.0,1.0,1.0,1.0,1.0])     |
|4  |The last document is the fifth one  |[the, last, document, is, the, fif

In [25]:
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [26]:
idf_w = idf.fit(hashing)

In [29]:
idf_w.fit(hashing).transform(hashing).show(truncate = False)

AttributeError: 'IDFModel' object has no attribute 'fit'